<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/HW7/hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505174 sha256=47b476901e39a399306f02275f3d7f114681170da4e5402f61243ef2e68f756c
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd

from surprise import accuracy, Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset

In [3]:
data = Dataset.load_builtin('ml-100k', prompt=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
data

In [5]:
# Отримуємо список рейтингів у вигляді списку кортежів (user, item, rating)
raw_ratings = data.raw_ratings

# Створюємо DataFrame
df = pd.DataFrame(raw_ratings, columns=["user_id", "item_id", "rating", "timestamp"])

# Виводимо перші 5 рядків
print(df.head())

  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596


---
# Використання алгоритму SVD
---

In [13]:
algo = SVD()

# Run 5-fold cross-validation and print results
result_svd = cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)
result_svd = pd.DataFrame.from_dict(result_svd).mean(axis=0)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9326  0.9333  0.9434  0.9321  0.9378  0.9358  0.0043  
MAE (testset)     0.7356  0.7357  0.7449  0.7337  0.7415  0.7383  0.0042  
Fit time          1.23    1.18    1.23    1.22    1.39    1.25    0.07    
Test time         0.09    0.21    0.08    0.29    0.09    0.15    0.09    


---
# Використання алгоритму SVD++
---

In [14]:
algoSVDpp = SVDpp()

resultSVDpp = cross_validate(algoSVDpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
resultSVDpp = pd.DataFrame.from_dict(resultSVDpp).mean(axis=0)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9190  0.9239  0.9201  0.9204  0.9194  0.9206  0.0018  
MAE (testset)     0.7193  0.7239  0.7254  0.7224  0.7194  0.7221  0.0024  
Fit time          25.51   25.64   25.55   26.90   25.87   25.89   0.52    
Test time         4.86    4.27    4.74    4.42    4.44    4.55    0.22    


---
# Використання алгоритму NMF
---

In [15]:
algoNMF = NMF()

resultNMF = cross_validate(algoNMF, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
resultNMF = pd.DataFrame.from_dict(resultNMF).mean(axis=0)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9646  0.9653  0.9603  0.9665  0.9674  0.9648  0.0025  
MAE (testset)     0.7596  0.7602  0.7499  0.7604  0.7625  0.7585  0.0044  
Fit time          2.27    2.59    2.25    2.30    2.26    2.34    0.13    
Test time         0.21    0.08    0.08    0.08    0.08    0.10    0.05    


---
# Підсумки
---

In [25]:
agg_results = pd.DataFrame({
    "SVD": result_svd,
    "SVD++": resultSVDpp,
    "NMF": resultNMF
})

print(agg_results)


                SVD      SVD++       NMF
test_rmse  0.935846   0.920561  0.964826
test_mae   0.738280   0.722107  0.758532
fit_time   1.247777  25.894733  2.336383
test_time  0.151258   4.546071  0.104412


# **Висновки**

<u>Якщо потрібна найкраща точність:</u> **→ SVD++**

✅ Найменші похибки (RMSE = 0.9206, MAE = 0.7221)

❌ Дуже повільне навчання (25.89 секунд) і тестування (4.55 секунд).


<u>Якщо потрібна швидкість і висока точність:</u> **→ SVD**

✅ Достатньо точний (RMSE = 0.9358, MAE = 0.7383)

✅ Швидке навчання (1.25 секунди) та тестування (0.15 секунди).

<u>Якщо точність не критична, але важлива інтерпретація:</u> **→ NMF**

✅ Модель легко інтерпретувати, оскільки базується на матричному розкладі.

❌ Найгірша точність (RMSE = 0.9648, MAE = 0.7585).

**Рекомендація**

Якщо ресурсів обмежено, виберіть SVD — він достатньо точний і швидкий.

Якщо найвища точність критична, використовуйте SVD++, але готуйтеся до довгого навчання.

NMF підходить лише, якщо потрібна зрозуміла модель, але вона менш точна.

---
# Тест передбачення
---

In [28]:
# Приклад: передбачення рейтингу для користувача '196' та фільму '242'
user_id = '196'
item_id = '242'

# Викликаємо метод predict() для отримання прогнозу
prediction = algo.predict(user_id, item_id)

# Виводимо передбачений рейтинг
print(f"Передбачений рейтинг для користувача {user_id} та фільму {item_id} - SVD: {prediction.est:.4f}")


# Для SVD++
prediction_svdpp = algoSVDpp.predict(user_id, item_id)
print(f"SVD++: Передбачений рейтинг: {prediction_svdpp.est:.4f}")

# Для NMF
prediction_nmf = algoNMF.predict(user_id, item_id)
print(f"NMF: Передбачений рейтинг: {prediction_nmf.est:.4f}")


# Фільтруємо DataFrame за користувачем і фільмом
actual_rating = df[(df["user_id"] == user_id) & (df["item_id"] == item_id)]

# Якщо запис знайдено, виводимо реальний рейтинг
if not actual_rating.empty:
    print(f"Реальний рейтинг користувача {user_id} для фільму {item_id}: {actual_rating['rating'].values[0]}")
else:
    print(f"Користувач {user_id} не оцінював фільм {item_id}")

Передбачений рейтинг для користувача 196 та фільму 242  SVD: 3.9609
SVD++: Передбачений рейтинг: 3.9189
NMF: Передбачений рейтинг: 4.1517
Реальний рейтинг користувача 196 для фільму 242: 3.0


---
# **Додаткове завдання з зірочкою**
---

Алгоритм колабораційної фільтрації